In [37]:
# Author: ehud@volcani.agri.gov.il

In [156]:
%reset -f

In [157]:
import os
import glob 
import pandas as pd
import numpy as np

In [158]:
# change the value of the variable `directory` to "control", "stage1", "stage2" or "stage3"
directory="stage2"
os.chdir(os.path.join(r"C:\Users\ehud\Documents\bell-pepper\2025-04-0609", directory)) # Set the current working directory
csv_files=glob.glob("*.csv")  # Find all .csv files in the directory "directory"

In [159]:
# from each csv file in the current working directory, read only the "Intensity" column and concatenate all columns to one dataframe, which we shall call `dataset`:
dataset=pd.concat(
    [
        pd.read_csv(file, usecols=["Intensity"]).rename(
            columns={"Intensity": os.path.splitext(file)[0]}
        )
        for file in csv_files
    ],
    axis=1
)
#dataset

In [160]:
# prepend to this dataframe the wavelength column and reset the result. The wv column is taken from one of the csv files that has already been read to the above dataframe.
# All wv columns in this directory should be identical; For convenience we take the column from the first file.

# Read the Wavelength column from the first csv file
wavelength = pd.read_csv(csv_files[0], usecols=["Wavelength"])


dataset=pd.concat([pd.read_csv(csv_files[0], usecols=["Wavelength"]).rename(columns={"Wavelength":"wv"}), dataset], axis=1)

In [161]:
dataset

,wv,001,002,003,004,005,006,007,008,009,...,111,112,113,114,115,116,117,118,119,120
0,804.404322,52180,47235,43751,44252,39627,44941,42384,39088,38435,...,44517,45024,45114,43667,45069,50907,48034,47418,50095,45171
1,804.674093,52921,47899,44381,44720,40282,45433,43367,39360,38523,...,45201,45369,46105,44422,45938,51344,48418,48017,50612,45743
2,804.943872,52943,48253,44967,45196,41027,45901,43812,39957,38746,...,46131,46009,46270,45253,46260,51604,49195,48376,51197,46244
3,805.213661,53568,48579,45113,45458,40745,46319,43624,39895,39413,...,46104,46296,46279,44792,46690,52130,49233,48922,51709,46442
4,805.483458,53362,48523,44972,45984,41070,46826,43691,39964,39490,...,46035,46119,46358,44925,46518,52397,49504,48662,51399,46689
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1019,1080.210034,2544,2789,4316,2468,2183,3434,4577,4010,1970,...,2100,3614,2023,2266,3121,4489,1868,1744,1820,2270
1020,1080.477879,2389,2737,4167,2447,2168,3378,4737,3985,1963,...,2073,3518,1974,2300,3013,4411,1778,1684,1850,2178
1021,1080.745711,2451,2724,4118,2437,2123,3348,4563,3971,1940,...,2128,3543,1927,2216,2971,4425,1718,1743,1740,2134
1022,1081.013530,2371,2753,4086,2431,2065,3368,4590,3929,1933,...,2199,3548,1914,2213,2932,4360,1739,1753,1807,2198


In [162]:
# Create a new dataframe with the value of numerical integral of spectral intensity vs. wavelength, for each column:
results = {}

for col in range(dataset.shape[1] - 1):
    col_name = dataset.columns[col + 1]
    integral = np.trapezoid(dataset.iloc[:, col + 1], dataset['wv'])
    results[col_name] = integral
# Convert the results to a DataFrame if needed
integrals = pd.DataFrame([results])
del results
del col_name
del integral
integrals


,001,002,003,004,005,006,007,008,009,010,...,111,112,113,114,115,116,117,118,119,120
0,7.409123e+06,7.015312e+06,7.885222e+06,6.739168e+06,6.088904e+06,7.115420e+06,8.090240e+06,6.977859e+06,6.564119e+06,7.390417e+06,...,6.465222e+06,6.811443e+06,6.308446e+06,6.455948e+06,6.687574e+06,7.827063e+06,6.820505e+06,7.250141e+06,6.730236e+06,6.577190e+06


In [163]:
# check that `openpyxl` is installed (pandas engine used to read xls files.
# pip install openpyxl
# read the correspondence file that matches measurement and its exposure time:
deltat = pd.read_excel(
    r"..\correspondence_exposure_times.xlsx",
    sheet_name=directory,
    dtype={"file name": str}  # Specify the column to be read as string
).iloc[:, -2:] # import only the last two columns in the excel sheet
deltat


,file name,exposure time (s)
0,001,3.0
1,002,3.0
2,003,5.0
3,004,2.5
4,005,2.0
...,...,...
115,116,4.0
116,117,1.1
117,118,1.0
118,119,1.1


In [ ]:
# Add a new row to the integrals dataset using values from the times column in `deltat`, matched by column names in `integrals` and filenames in deltat.
# Match the exposure times from deltat with the column names in integrals

##### This cell doesn't work for some reason

# Match the exposure times from deltat with the column names in integrals
exposure_times = deltat.set_index('file name')['exposure time (s)']

# Add the exposure times as a new row to the integrals DataFrame
integrals.loc[1] = exposure_times.values
integrals

001   NaN
002   NaN
003   NaN
004   NaN
005   NaN
       ..
116   NaN
117   NaN
118   NaN
119   NaN
120   NaN
Name: exposure time (s), Length: 120, dtype: float64

In [172]:
# Divide the first row (propto power) by the second row (exposure time) to get a measure for the energy arriving to the sensor
integrals.loc[2] = integrals.loc[0] / integrals.loc[1]
integrals

,001,002,003,004,005,006,007,008,009,010,...,111,112,113,114,115,116,117,118,119,120
0,7.409123e+06,7.015312e+06,7.885222e+06,6.739168e+06,6.088904e+06,7.115420e+06,8.090240e+06,6.977859e+06,6.564119e+06,7.390417e+06,...,6.465222e+06,6.811443e+06,6.308446e+06,6.455948e+06,6.687574e+06,7.827063e+06,6.820505e+06,7.250141e+06,6.730236e+06,6.577190e+06
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [132]:
# Calculate the mean of the control group (mean of the last row in `integrals`)
average_energy = integrals.loc[2].mean()

In [133]:
# Write the result to a text file in the current working directory
with open('..\\average_energy.txt', 'a') as f:
    f.write(f"{directory}\t{average_energy}\n")